In [62]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

In [25]:
import sys
sys.path.append('../')
import utils

In [80]:
# Load the dataset
df = pd.read_csv('../data/illinois_basing_train.csv')
# Rename target column
df = df.rename(columns={'inj_diff\xa0': 'Target', 'SampleTimeUTC': 'Date'})

# Change Target outliers to regular points
df.at[836, 'Target'] = 15
df.at[837, 'Target'] = 30
df.at[838, 'Target'] = -44.5
df.at[839, 'Target'] = -0.5

In [81]:
df = df.dropna(subset='Target')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df = df.set_index('Date').sort_index()

In [82]:
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
missing_hours = date_range.difference(df.index)
if not missing_hours.empty:
    print("Missing hours found:")
    print(missing_hours)
else:
    print("No missing hours found.")

Missing hours found:
DatetimeIndex(['2010-01-31 00:00:00', '2010-01-31 01:00:00',
               '2010-01-31 02:00:00', '2010-01-31 03:00:00',
               '2010-03-14 06:00:00', '2010-03-14 07:00:00',
               '2010-03-14 08:00:00', '2010-04-12 16:00:00',
               '2010-04-12 17:00:00', '2010-04-12 18:00:00',
               '2010-04-12 19:00:00', '2010-04-12 20:00:00',
               '2010-04-12 21:00:00', '2010-04-12 22:00:00',
               '2010-04-12 23:00:00', '2010-04-13 00:00:00',
               '2010-04-13 01:00:00', '2010-04-13 02:00:00',
               '2010-04-13 03:00:00', '2010-04-13 04:00:00',
               '2010-04-13 05:00:00', '2010-04-13 06:00:00',
               '2010-04-13 07:00:00', '2010-05-09 15:00:00',
               '2010-09-25 22:00:00', '2010-09-25 23:00:00',
               '2010-09-26 00:00:00', '2010-09-26 01:00:00',
               '2010-09-26 02:00:00', '2011-01-16 16:00:00',
               '2011-01-16 17:00:00', '2011-01-16 18:00:00',
   

In [83]:
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
df = df.reindex(date_range, fill_value=np.nan)
df.reset_index(inplace=True, drop=False)
df = df.rename(columns={'index': 'Date'})

In [84]:
new_row_indices = df[df['Target'].isnull()].index
new_row_indices

Int64Index([ 2927,  2928,  2929,  2930,  3941,  3942,  3943,  4647,  4648,
             4649,  4650,  4651,  4652,  4653,  4654,  4655,  4656,  4657,
             4658,  4659,  4660,  4661,  4662,  5294,  8637,  8638,  8639,
             8640,  8641, 11343, 11344, 11345, 11346, 13244, 13245, 13246,
            13247, 19519, 19520, 19521, 19522, 19523, 19524, 19525, 19526,
            24470, 24471, 24472, 24473, 24474, 24475, 24476, 24477, 24478,
            24479, 24480, 24481, 24482, 24483, 24484, 24485, 24486, 24487,
            24488, 24489, 24490],
           dtype='int64')

In [108]:
l1 = [2925, 2926, 2927,  2928,  2929,  2930, 2931, 2932]

l2 = [ 3941,  3942,  3943]
l3 = [4647,  4648, 4649,  4650,  4651,  4652,  4653,  4654,  4655,  4656,  4657,
             4658,  4659,  4660,  4661,  4662]
l4 = [5294]
l5 = [ 8637,  8638,  8639, 8640,  8641]
l6 = [11343, 11344, 11345, 11346]
l7 = [13244, 13245, 13246,
            13247]
l8 = [19519, 19520, 19521, 19522, 19523, 19524, 19525, 19526]
l9 = [24470, 24471, 24472, 24473, 24474, 24475, 24476, 24477, 24478,
            24479, 24480, 24481, 24482, 24483, 24484, 24485, 24486, 24487,
            24488, 24489, 24490]

In [111]:
import pickle 
with open('../metadata/missing_date_idxs.pkl', 'wb') as f:
    pickle.dump(new_row_indices, f)

In [90]:
from itertools import groupby
from operator import itemgetter

In [96]:
import pandas as pd
import numpy as np
from itertools import groupby
from sklearn.linear_model import LinearRegression

# Define the window size for linear regression
window_size = 5

# Group the consecutive missing indices into windows
windows = []
for k, g in groupby(enumerate(new_row_indices), lambda ix: ix[0] - ix[1]):
    windows.append(list(map(itemgetter(1), g)))

# Loop over the columns
for col in df.columns:
    # Loop over the windows
    for window in windows:
        # Get the indices of non-missing values for this column
        start = max(window[0] - window_size, 0)
        end = min(window[-1] + window_size + 1, len(df))
        non_missing = df.loc[start:end, col].dropna()
        
        # Fit a linear regression using the previous 5 and next 5 non-missing values
        if len(non_missing) >= window_size * 2:
            X = np.array(non_missing.index).reshape(-1, 1)
            y = non_missing.values.reshape(-1, 1)
            model = LinearRegression()
            model.fit(X, y)
            
            # Predict the missing values
            for i in window:
                if pd.isna(df.loc[i, col]):
                    y_pred = model.predict([[i]])[0][0]
                    # Fill the missing value with the predicted value
                    df.loc[i, col] = y_pred


In [107]:
for col in df.columns:
    if col not in df.columns:
        continue
    print(col)
    print(df[col].loc[l2])

Date
3941   2010-03-14 06:00:00
3942   2010-03-14 07:00:00
3943   2010-03-14 08:00:00
Name: Date, dtype: datetime64[ns]
Avg_PLT_CO2VentRate_TPH
3941    0.0
3942    0.0
3943    0.0
Name: Avg_PLT_CO2VentRate_TPH, dtype: float64
Avg_CCS1_WHCO2InjPs_psi
3941    789.769316
3942    662.125714
3943    534.482111
Name: Avg_CCS1_WHCO2InjPs_psi, dtype: float64
Avg_CCS1_WHCO2InjTp_F
3941    59.031042
3942    49.498308
3943    39.965575
Name: Avg_CCS1_WHCO2InjTp_F, dtype: float64
Avg_CCS1_ANPs_psi
3941    397.363115
3942    332.974558
3943    268.586001
Name: Avg_CCS1_ANPs_psi, dtype: float64
Avg_CCS1_DH6325Ps_psi
3941    3269.427072
3942    3269.627138
3943    3269.827204
Name: Avg_CCS1_DH6325Ps_psi, dtype: float64
Avg_CCS1_DH6325Tp_F
3941    121.986216
3942    122.568001
3943    123.149786
Name: Avg_CCS1_DH6325Tp_F, dtype: float64
Avg_VW1_WBTbgPs_psi
3941   NaN
3942   NaN
3943   NaN
Name: Avg_VW1_WBTbgPs_psi, dtype: float64
Avg_VW1_WBTbgTp_F
3941   NaN
3942   NaN
3943   NaN
Name: Avg_VW1_WBTbgTp

In [45]:
df['Abs Target'] = np.abs(df['Target'])
df['Abs Target'] = df['Abs Target'].clip(upper=45.7)

windows = utils.create_fluctuation_windows(df, threshold=2, target_column_name='Target', date_column_name='Date')
df = df.set_index('Date').sort_index()

# Create label for +/- 2
df['Target > 2'] = df['Abs Target'].apply(lambda x: 1 if x > 2 else 0)

# Create new label to predict start of window
df['window_start'] = 0
for window_indices, _ in windows:
    df.loc[window_indices[0], 'window_start'] = 1

# Create new label to predict duration of window
df['window_duration'] = 0
for window_indices, _ in windows:
    df.loc[window_indices[0], 'window_duration'] = len(window_indices)

# Create a new column called 'within_window' with all values set to 0
df['within_window'] = 0

# Set the corresponding 'within_window' value to 1 for each row within the windows
for window_indices, _ in windows:
    df.loc[df.index.isin(window_indices), 'within_window'] = 1

In [46]:
df.reset_index(inplace=True, drop=False)
df = df.rename(columns={'index': 'Date'})

In [100]:
df['Target'].isna().sum()

0

In [49]:
df['Avg_VW1_Z03D6945Tp_F'] = df['Avg_VW1_Z03D6945Tp_F'].clip(upper=128.5)
df.at[836, 'Target'] = 15
df.at[837, 'Target'] = 30
df.at[838, 'Target'] = -44.5
df.at[839, 'Target'] = -0.5

df.loc[834, 'Avg_PLT_CO2VentRate_TPH'] = 0
df.loc[835, 'Avg_PLT_CO2VentRate_TPH'] = 0
df.loc[836, 'Avg_PLT_CO2VentRate_TPH'] = 0

In [2]:
# df = pd.read_csv('../data/train_df_with_labels.csv')
# df = df.drop(['Date', 'Target', 'Abs Target', 'window_start','window_duration', 'within_window'], axis=1)

In [50]:
df.head(2)

,Date,Avg_PLT_CO2VentRate_TPH,Avg_CCS1_WHCO2InjPs_psi,Avg_CCS1_WHCO2InjTp_F,Avg_CCS1_ANPs_psi,Avg_CCS1_DH6325Ps_psi,Avg_CCS1_DH6325Tp_F,Avg_VW1_WBTbgPs_psi,Avg_VW1_WBTbgTp_F,Avg_VW1_ANPs_psi,...,Avg_VW1_Z01D7061Ps_psi,Avg_VW1_Z01D7061Tp_F,Avg_VW1_Z0910D5482Ps_psi,Avg_VW1_Z0910D5482Tp_F,Target,Abs Target,Target > 2,window_start,window_duration,within_window
0,2009-10-01 01:00:00,20.543221,1.374349,55.654541,89.825334,2893.793620,116.538811,2173.762679,104.049292,1599.975952,...,3216.520127,120.276792,2442.006201,111.891938,0.0,0.0,0,0,0,0
1,2009-10-01 02:00:00,20.543221,1.315104,53.661254,89.806754,2893.791506,116.538623,2173.754085,104.050357,1599.975952,...,3216.510374,120.280932,2442.070968,111.891218,0.0,0.0,0,0,0,0


In [33]:
target = 'Target > 2'

In [34]:
# Define the range of k values to test
k_values = list(range(1, 8))

# Initialize KFold cross-validation
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# Function to calculate cross-validation scores for different k values
def knn_imputation_cv(df, k_values, target, kf):
    scores = []

    for k in k_values:
        imputer = KNNImputer(n_neighbors=k)

        # Create a pipeline with StandardScaler and LogisticRegression
        pipeline = Pipeline([
            # ('scaler', StandardScaler()),
            ('classifier', DecisionTreeClassifier())
        ])


        # Perform cross-validation for the current k value
        score = np.average(
            cross_val_score(
                pipeline,
                imputer.fit_transform(df.drop(target, axis=1)),
                df[target],
                cv=kf,
                scoring=make_scorer(average_precision_score)
            )
        )
        scores.append(score)

    return scores

# Calculate cross-validation scores for each k value
cv_scores = knn_imputation_cv(df, k_values, target, kf)

# Find the best k value
best_k = k_values[np.argmax(cv_scores)]
print(f"Best value of k: {best_k}")

Best value of k: 4


In [36]:
cv_scores

[0.25154529414999943,
 0.2631354715403432,
 0.25974006032521124,
 0.270013253262668,
 0.265618858168397,
 0.26673052242820927,
 0.2657803161706597]

In [51]:
# Impute using K neighours found in the other notebook
imputer = KNNImputer(n_neighbors=4, weights='uniform')

y = df[['Date', 'window_start', 'window_duration', 'within_window', 'Target', 'Abs Target', 'Target > 2']]
x = df.drop(['Date', 'window_start', 'window_duration', 'within_window', 'Target', 'Abs Target', 'Target > 2'], axis=1)
cols = x.columns

x = imputer.fit_transform(x)
x = pd.DataFrame(x, columns=cols)

In [87]:
for col in df.columns:
    if col not in df.columns:
        continue
    print(col)
    print(df[col].loc[l1])

Date
2925   2010-01-30 22:00:00
2926   2010-01-30 23:00:00
2927   2010-01-31 00:00:00
2928   2010-01-31 01:00:00
2929   2010-01-31 02:00:00
2930   2010-01-31 03:00:00
2931   2010-01-31 04:00:00
2932   2010-01-31 05:00:00
Name: Date, dtype: datetime64[ns]
Avg_PLT_CO2VentRate_TPH
2925    0.0
2926    0.0
2927    NaN
2928    NaN
2929    NaN
2930    NaN
2931    0.0
2932    0.0
Name: Avg_PLT_CO2VentRate_TPH, dtype: float64
Avg_CCS1_WHCO2InjPs_psi
2925    1132.004229
2926    1126.026978
2927            NaN
2928            NaN
2929            NaN
2930            NaN
2931    1116.058944
2932    1115.850574
Name: Avg_CCS1_WHCO2InjPs_psi, dtype: float64
Avg_CCS1_WHCO2InjTp_F
2925    80.902016
2926    80.338901
2927          NaN
2928          NaN
2929          NaN
2930          NaN
2931    80.653358
2932    80.685951
Name: Avg_CCS1_WHCO2InjTp_F, dtype: float64
Avg_CCS1_ANPs_psi
2925    576.251348
2926    564.369290
2927           NaN
2928           NaN
2929           NaN
2930           NaN
2931   

In [21]:
# Define the range of k values to test
k_values = list(range(1, 10))

# Initialize KFold cross-validation
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# Function to calculate cross-validation scores for different k values
def knn_imputation_cv(df, k_values, target, kf):
    scores = []

    for k in k_values:
        imputer = KNNImputer(n_neighbors=k, weights='distance')

        # Create a pipeline with StandardScaler and LogisticRegression
        pipeline = Pipeline([
            # ('scaler', StandardScaler()),
            ('classifier', DecisionTreeClassifier())
        ])


        # Perform cross-validation for the current k value
        score = np.average(
            cross_val_score(
                pipeline,
                imputer.fit_transform(df.drop(target, axis=1)),
                df[target],
                cv=kf,
                scoring=make_scorer(average_precision_score)
            )
        )
        scores.append(score)

    return scores

# Calculate cross-validation scores for each k value
cv_scores = knn_imputation_cv(df, k_values, target, kf)

# Find the best k value
best_k = k_values[np.argmax(cv_scores)]
print(f"Best value of k: {best_k}")

Best value of k: 3


In [22]:
cv_scores

[0.24914225901811335,
 0.25557975590486576,
 0.2696763524983467,
 0.25404307524988146,
 0.2543403632424305,
 0.2493044974856226,
 0.2674413833938519,
 0.26053275353243627,
 0.2642490013444993]